In [24]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"   
from sklearn.metrics import roc_auc_score
import argparse as ap
from sklearn import preprocessing
import sys
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, Dense, Flatten, MaxPooling2D
from numpy import *
from sklearn import *
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed
import time
from ete3 import NCBITaxa

In [25]:
DICT = {}
for line in open("abundance.csv"):
    split = line.split("\t")
    otu = split[0].split("g__")
#     print(otu)
    if len(otu) > 1:
        otu = otu[1]
    else:
        otu = otu[0].split("f__")[1]
    otu = otu.split("_noname")[0].split("_unclassified")[0].replace("_", " ").replace("XIII", "XIII.").strip()
    abun = np.array(split[1:], dtype=float)
    if otu in DICT:
        DICT[otu] = np.add(DICT[otu], abun)
    else:
        DICT[otu] = abun
otus = DICT.keys()
abuns = np.vstack(DICT.values()).T
# print(len(otus))
X = pd.DataFrame(data=abuns, columns=otus)
X

/tmp/ipykernel_3051753/1855423559.py:17: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  abuns = np.vstack(DICT.values()).T


,Methanobrevibacter,Methanosphaera,Acidobacteriaceae,Actinomyces,Varibaculum,Rothia,Bifidobacterium,Gardnerella,Adlercreutzia,Atopobium,...,Rheinheimera,Rhodanobacter,Xanthomonas,Microlunatus,Meiothermus,Shinella,Pseudoalteromonas,Vibrio,Zunongwangia,Rhodopirellula
0,0.33364,0.0,0.0,0.00000,0.0,0.00000,3.86983,0.00000,0.05426,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.62579,0.0,0.0,0.00000,0.0,0.00000,0.00284,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.00000,0.0,0.01515,0.41549,0.00283,0.00031,0.00625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.00000,0.0,0.02847,0.03583,0.00344,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.56233,0.0,0.0,0.00000,0.0,0.02221,0.12154,0.00000,0.01488,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,0.02598,0.0,0.0,0.00000,0.0,0.00000,31.85710,0.00000,0.03083,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
436,2.78607,0.0,0.0,0.00000,0.0,0.03371,0.00000,0.00000,0.19822,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
437,2.46789,0.0,0.0,0.00000,0.0,0.00000,9.64555,0.00000,0.06288,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438,6.79589,0.0,0.0,0.03085,0.0,0.00000,1.46966,0.00000,0.04413,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn import preprocessing
y = pd.read_csv('labels.txt',header=None)
le = preprocessing.LabelEncoder()
y=np.array(y).ravel()
y = le.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [27]:
raw_name=X.columns.values.tolist()
ncbi = NCBITaxa()
raw_id = ncbi.get_name_translator(raw_name)
raw_id = [str(i[0]) for i in list(raw_id.values())]
X = pd.DataFrame(data=abuns, columns=raw_id)
len(raw_id)
# tree = ncbi.get_topology(raw_id)
# print (tree.get_ascii(attributes=["taxid"]))

214

In [28]:
# Also, we can use the Newick obtained file to get the tree by using PhyloT, just like the
# description in our paper
tree = ncbi.get_topology(raw_id)
print (tree.get_ascii(attributes=["taxid"]))

order = []
num = 1
for node in tree.traverse(strategy='levelorder'):
    if node.is_leaf():
        order.append(node.name)

postorder = []
num = 1
for node in tree.traverse(strategy='postorder'):
    if node.is_leaf():
        postorder.append(node.name)

temp = []
for i in order:
    if i in X.columns:
        temp.append(i)

order = temp

temp1 = []
for i in postorder:
    if i in X.columns:
        temp1.append(i)

postorder  = temp1

known_Xl=X[order]
known_Xp=X[postorder]

known_Xl.to_csv('knownl.csv')
known_Xp.to_csv('knownp.csv')
# for unknown features, we just arrange the taxa with at least genus levels.


                           /-32257
                        /481
                  /206351  \-482
                 |     |
                 |      \-535
                 |
                 |           /-47670
                 |          |
                 |     /119060-106589
                 |    |     |
                 |    |      \-32008
                 |    |
                 |    |      /-40544
                 |    |-995019
                 |    |      \-577310
             /28216   |
            |    |-80840    /-52972
            |    |    |    |
            |    |    |    |--665874
            |    |    |-80864
            |    |    |    |--34072
            |    |    |    |
            |    |    |     \-283
            |    |    |
            |    |    |     /-846
            |    |     \75682
            |    |          \-884214
            |    |
            |     \-90627
            |
            |          /267888-53246
            |    /135622
            |   |      \-

In [29]:
knownl = pd.read_csv('knownl.csv', index_col=0)
knownp = pd.read_csv('knownp.csv', index_col=0)

In [30]:
knownl

,95818,558314,29521,196117,5073,2093,265488,239934,1091,940557,...,580596,1257,1578,46255,1243,1253,1301,1357,400634,1386
0,0.00000,0.02201,0.35353,0.0,0.0,0.0,0.0,0.00000,0.18612,0.0,...,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000
1,0.00000,0.00000,0.14902,0.0,0.0,0.0,0.0,0.00000,0.08670,0.0,...,0.00357,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00092,0.00000
2,0.00000,0.00000,7.58386,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.01704,0.00000
3,0.00000,0.00000,0.32546,0.0,0.0,0.0,0.0,0.01253,0.03792,0.0,...,0.00000,0.0,0.0,0.0,0.00435,0.0,0.0,0.00656,0.00323,0.00000
4,0.02521,0.00733,0.05261,0.0,0.0,0.0,0.0,0.00000,0.15515,0.0,...,0.00000,0.0,0.0,0.0,0.00599,0.0,0.0,0.00000,0.00000,0.00285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,0.00000,0.00000,0.34611,0.0,0.0,0.0,0.0,0.00000,0.01242,0.0,...,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000
436,3.18370,0.00000,0.38420,0.0,0.0,0.0,0.0,0.00000,0.14179,0.0,...,0.00175,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000
437,0.00000,0.00282,1.48540,0.0,0.0,0.0,0.0,0.00000,0.19392,0.0,...,0.01592,0.0,0.0,0.0,0.01404,0.0,0.0,0.00000,0.00000,0.00000
438,13.01539,0.02607,1.62649,0.0,0.0,0.0,0.0,0.00000,0.02370,0.0,...,0.01794,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000


In [31]:
def transform_level(X):
    X = np.array(X)
    raw_dim = X.shape[1]
    img_size = math.ceil(raw_dim ** 0.5)
    new_dim = img_size ** 2

    add_blank = np.zeros((X.shape[0], new_dim - raw_dim))
    new_X = np.hstack((X, add_blank))
    new_X = new_X.reshape(X.shape[0], img_size, -1)
    print(new_X.shape)

    base_log = 4
    new_X = np.log(new_X + 1) / np.log(base_log)

    bins_break = [[0.0065536, np.max(new_X)],
                  [0.0016384, 0.0065536],
                  [0.0004096, 0.0016384],
                  [0.0001024, 0.0004096],
                  [0.0000256, 0.0001024],
                  [0.0000064, 0.0000256],
                  [0.0000016, 0.0000064],
                  [0.0000004, 0.0000016],
                  [0.0000001, 0.0000004],
                  [0, 0.0000001]]

    color_arry_num = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    for windows in range(10):
        index = (new_X >= bins_break[windows][0]) & (new_X < bins_break[windows][1])
        new_X[index] = color_arry_num[windows]

    return new_X

def transform_post(X):
    X = np.array(X)
    raw_dim = X.shape[1]
    img_size = math.ceil(raw_dim ** 0.5)
    new_dim = img_size ** 2

    add_blank = np.zeros((X.shape[0], new_dim - raw_dim))
    new_X = np.hstack((X, add_blank))
    new_X = new_X.reshape(X.shape[0], img_size, -1)

    for img in new_X:
        for line in range(img.shape[0]):
            if line % 2 != 0:
                img[line] = img[line][::-1]
    print(new_X.shape)

    base_log = 4
    new_X = np.log(new_X + 1) / np.log(base_log)

    bins_break = [[0.0065536, np.max(new_X)],
                  [0.0016384, 0.0065536],
                  [0.0004096, 0.0016384],
                  [0.0001024, 0.0004096],
                  [0.0000256, 0.0001024],
                  [0.0000064, 0.0000256],
                  [0.0000016, 0.0000064],
                  [0.0000004, 0.0000016],
                  [0.0000001, 0.0000004],
                  [0, 0.0000001]]

    color_arry_num = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    for windows in range(10):
        index = (new_X >= bins_break[windows][0]) & (new_X < bins_break[windows][1])
        new_X[index] = color_arry_num[windows]

    return new_X


In [32]:
# par = read_params(sys.argv)
# file_name = str(par['fn'])
# ts = float(par['ts'])
# rs = int(par['rs'])
# et = str(par['et'])
et='t' 
ts = 0.3 
rs = 10

if et=='t':
    earlystop = EarlyStopping(monitor='loss',
                              min_delta=1e-4,
                              patience=10,
                              verbose=1)
else:
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=1e-4,
                              patience=10,
                              verbose=1)

def train_model(X_train, y_train, X_test, epochs, earlystop, et):
    K.clear_session()

    model = Sequential()
    model.add(Convolution2D(input_shape=(X_train.shape[1], X_train.shape[1], 1), kernel_size=(5, 5), filters=20,
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

    model.add(Convolution2D(kernel_size=(5, 5), filters=50, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    if et=='t':

        model.fit(X_train, y_train,
                  batch_size=X_train.shape[0],
                  epochs=epochs,
                  callbacks=[earlystop],
                  shuffle=True)

    else:
        model.fit(X_train, y_train,
                  batch_size=X_train.shape[0],
                  epochs=epochs,
                  validation_split=0.1,
                  callbacks=[earlystop],
                  shuffle=True)

    pred = model.predict(X_test)[:, 1]
    return pred

In [33]:
knownl=transform_level(knownl)
knownp=transform_post(knownp)

knownl= np.expand_dims(knownl, axis=3)
knownp = np.expand_dims(knownp, axis=3)

start = time.time()

new_XL_train, new_XL_test, new_XP_train, new_XP_test, \
y_train, y_test = train_test_split(knownl, knownp, y, test_size=ts, random_state=4489)

y_train = to_categorical(y_train, num_classes=2)

print('++++++++++++++++++++++')
print(new_XL_train.shape)

(440, 14, 14)
(440, 14, 14)
++++++++++++++++++++++
(308, 14, 14, 1)


In [34]:
ave_auc = []
ave_auc1 = []
ave_auc2 = []
ave_auc12 = []
repeat_seed = rs

for i in range(repeat_seed):
    K.clear_session()
    seed(i)
    tensorflow.random.set_seed(i)
    epochs = 500
    predL = train_model(new_XL_train, y_train, new_XL_test, epochs, earlystop, et)
    predP = train_model(new_XP_train, y_train, new_XP_test, epochs, earlystop, et)
    pred = (predL + predP) / 2
    pred1 = (predL)
    pred2 = (predP)
    pred12 = (predL + predP) / 2
    auc = roc_auc_score(y_test, pred)
    auc1 = roc_auc_score(y_test, pred1)
    auc2 = roc_auc_score(y_test, pred2)
    auc12 = roc_auc_score(y_test, pred12)
    # print('AUC for ' + str(i + 1) + ' rounds:  %.4f' % auc)
    ave_auc.append(auc)
    ave_auc1.append(auc1)
    ave_auc2.append(auc2)
    ave_auc12.append(auc12)
    
# print(ave_auc)
# print('EPCNN Mean AUC :  %.4f' % mean(ave_auc))
mauc=mean(ave_auc)
# print(ave_auc1)
print('KnownL Mean AUC :  %.4f' % mean(ave_auc1))
mauc1=mean(ave_auc1)
# print(ave_auc2)
print('KnownP Mean AUC :  %.4f' % mean(ave_auc2))
mauc2=mean(ave_auc2)
print('EnKnownL Mean AUC :  %.4f' % mean(ave_auc12))
mauc12=mean(ave_auc12)
# print(ave_auc34)
end = time.time()
running_time = end - start
print(  'The best AUC of our EPCNN is: %.5f ' % max([mean(ave_auc),mean(ave_auc1),mean(ave_auc2),mean(ave_auc12)]) )
print('All Time cost : %.5f s' % running_time)

path = 'EPCNN_ev.txt'
if os.path.exists(path):
    os.remove(path)
file = open(path, 'a')
file.write('EPCNN Mean AUCs: ' + str(mauc) + "\n")
file.write('KnownL Mean AUCs: ' + str(mauc1) + "\n")
file.write('KnownP Mean AUCs: ' + str(mauc2) + "\n")
file.write('EnKnown Mean AUCs: ' + str(mauc12) + "\n")
file.write('Time for ' + str(repeat_seed) + ' rounds running: ' + str(running_time) + "s")

Epoch 1/500
1/1 [==============================] - 0s 207ms/step - loss: 0.6947 - accuracy: 0.4935
Epoch 2/500
1/1 [==============================] - 0s 11ms/step - loss: 0.7031 - accuracy: 0.5065
Epoch 3/500
1/1 [==============================] - 0s 11ms/step - loss: 0.6903 - accuracy: 0.5065
Epoch 4/500
1/1 [==============================] - 0s 11ms/step - loss: 0.6902 - accuracy: 0.4935
Epoch 5/500
1/1 [==============================] - 0s 14ms/step - loss: 0.6910 - accuracy: 0.4935
Epoch 6/500
1/1 [==============================] - 0s 12ms/step - loss: 0.6884 - accuracy: 0.4935
Epoch 7/500
1/1 [==============================] - 0s 11ms/step - loss: 0.6853 - accuracy: 0.6461
Epoch 8/500
1/1 [==============================] - 0s 11ms/step - loss: 0.6833 - accuracy: 0.6364
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 0.6817 - accuracy: 0.5747
Epoch 10/500
1/1 [==============================] - 0s 11ms/step - loss: 0.6793 - accuracy: 0.5812
Epoch 11/500
1/1 [

1/1 [==============================] - 0s 11ms/step - loss: 0.0919 - accuracy: 0.9968
Epoch 84/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0859 - accuracy: 0.9935
Epoch 85/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0798 - accuracy: 0.9968
Epoch 86/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0772 - accuracy: 0.9968
Epoch 87/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0724 - accuracy: 0.9968
Epoch 88/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0670 - accuracy: 0.9968
Epoch 89/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0645 - accuracy: 1.0000
Epoch 90/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0604 - accuracy: 0.9968
Epoch 91/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0561 - accuracy: 1.0000
Epoch 92/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0537 - accuracy: 1.0000
Epoch 93/500
1/1 [=====

1/1 [==============================] - 0s 12ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 166/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 167/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 168/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 169/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 170/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 171/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 172/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 173/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 174/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 175/500


1/1 [==============================] - 0s 10ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 248/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 249/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 250/500
1/1 [==============================] - 0s 18ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 251/500
1/1 [==============================] - 0s 12ms/step - loss: 9.9764e-04 - accuracy: 1.0000
Epoch 252/500
1/1 [==============================] - 0s 12ms/step - loss: 9.8539e-04 - accuracy: 1.0000
Epoch 253/500
1/1 [==============================] - 0s 12ms/step - loss: 9.7361e-04 - accuracy: 1.0000
Epoch 254/500
1/1 [==============================] - 0s 12ms/step - loss: 9.6182e-04 - accuracy: 1.0000
Epoch 255/500
1/1 [==============================] - 0s 12ms/step - loss: 9.5022e-04 - accuracy: 1.0000
Epoch 256/500
1/1 [==============================] - 0s 12ms/step - loss: 9.3882e-04 - accurac

1/1 [==============================] - 0s 11ms/step - loss: 0.2824 - accuracy: 0.9091
Epoch 50/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2745 - accuracy: 0.9156
Epoch 51/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2616 - accuracy: 0.9221
Epoch 52/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2485 - accuracy: 0.9318
Epoch 53/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2399 - accuracy: 0.9351
Epoch 54/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2345 - accuracy: 0.9351
Epoch 55/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2294 - accuracy: 0.9448
Epoch 56/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2200 - accuracy: 0.9545
Epoch 57/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2071 - accuracy: 0.9545
Epoch 58/500
1/1 [==============================] - 0s 11ms/step - loss: 0.1954 - accuracy: 0.9513
Epoch 59/500
1/1 [=====

...

1/1 [==============================] - 0s 10ms/step - loss: 0.5829 - accuracy: 0.7597
Epoch 28/500
1/1 [==============================] - 0s 10ms/step - loss: 0.5782 - accuracy: 0.6948
Epoch 29/500
1/1 [==============================] - 0s 10ms/step - loss: 0.5688 - accuracy: 0.7597
Epoch 30/500
1/1 [==============================] - 0s 10ms/step - loss: 0.5536 - accuracy: 0.7662
Epoch 31/500
1/1 [==============================] - 0s 10ms/step - loss: 0.5527 - accuracy: 0.7338
Epoch 32/500
1/1 [==============================] - 0s 10ms/step - loss: 0.5384 - accuracy: 0.7727
Epoch 33/500
1/1 [==============================] - 0s 12ms/step - loss: 0.5320 - accuracy: 0.7565
Epoch 34/500
1/1 [==============================] - 0s 12ms/step - loss: 0.5252 - accuracy: 0.7565
Epoch 35/500
1/1 [==============================] - 0s 12ms/step - loss: 0.5121 - accuracy: 0.7825
Epoch 36/500
1/1 [==============================] - 0s 12ms/step - loss: 0.5090 - accuracy: 0.7630
Epoch 37/500
1/1 [=====

1/1 [==============================] - 0s 11ms/step - loss: 0.0481 - accuracy: 0.9968
Epoch 110/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0459 - accuracy: 0.9968
Epoch 111/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0438 - accuracy: 0.9968
Epoch 112/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0415 - accuracy: 0.9968
Epoch 113/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0392 - accuracy: 0.9968
Epoch 114/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0372 - accuracy: 0.9968
Epoch 115/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0353 - accuracy: 0.9968
Epoch 116/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0333 - accuracy: 0.9968
Epoch 117/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0314 - accuracy: 0.9968
Epoch 118/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0296 - accuracy: 0.9968
Epoch 119/500


1/1 [==============================] - 0s 12ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 192/500
1/1 [==============================] - 0s 26ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 193/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 194/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 195/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 196/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 197/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 198/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 199/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 200/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 201/500


1/1 [==============================] - 0s 10ms/step - loss: 9.2775e-04 - accuracy: 1.0000
Epoch 273/500
1/1 [==============================] - 0s 10ms/step - loss: 9.1840e-04 - accuracy: 1.0000
Epoch 274/500
1/1 [==============================] - 0s 13ms/step - loss: 9.0930e-04 - accuracy: 1.0000
Epoch 275/500
1/1 [==============================] - 0s 12ms/step - loss: 9.0033e-04 - accuracy: 1.0000
Epoch 276/500
1/1 [==============================] - 0s 12ms/step - loss: 8.9138e-04 - accuracy: 1.0000
Epoch 277/500
1/1 [==============================] - 0s 12ms/step - loss: 8.8279e-04 - accuracy: 1.0000
Epoch 278/500
1/1 [==============================] - 0s 12ms/step - loss: 8.7408e-04 - accuracy: 1.0000
Epoch 279/500
1/1 [==============================] - 0s 11ms/step - loss: 8.6554e-04 - accuracy: 1.0000
Epoch 280/500
1/1 [==============================] - 0s 12ms/step - loss: 8.5722e-04 - accuracy: 1.0000
Epoch 281/500
1/1 [==============================] - 0s 12ms/step - loss: 8.48

46